<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


Коэффициент успешности запуска может зависеть от многих факторов, таких как масса полезной нагрузки, тип орбиты и так далее. Он также может зависеть от расположения и близости стартовой площадки, то есть от начального положения траектории ракеты. Поиск оптимального места для строительства космодрома, безусловно, включает в себя множество факторов, и мы надеемся, что сможем выявить некоторые из них, проанализировав существующие места расположения космодромов.

В предыдущих лабораторных работах по анализу исследовательских данных вы визуализировали набор данных о запусках SpaceX с помощью `matplotlib` и `seaborn` и обнаружили некоторые предварительные корреляции между местом запуска и показателями успешности. В этой лаборатории вам предстоит выполнить более интерактивную визуальную аналитику с помощью `Folium`.

## Цели


Эта лабораторная содержит следующие задания:
- **Задание 1:** Отметьте все места запуска на карте
- **Задание 2:** Отметьте на карте успешные/неудачные запуски для каждой площадки
- **Задание 3:** Рассчитайте расстояния от места запуска до ближайших к нему объектов

После выполнения перечисленных выше заданий вы должны уметь находить некоторые географические закономерности в отношении стартовых площадок.

Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=05677e031ee971a7cac120986a06006b6774e8be90cce6e4489af3777b713f9a
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

Если вам нужно освежить в памяти информацию о фолиуме, вы можете скачать и обратиться к этой предыдущей лабораторной работе по фолиуму:

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Задача 1: Отметьте на карте все стартовые площадки

Сначала попробуем добавить местоположение каждого сайта на карту, используя координаты широты и долготы сайта

Следующий набор данных с именем `spacex_launch_geo.csv` - это расширенный набор данных с широтой и долготой, добавленной для каждого сайта.

In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Теперь вы можете посмотреть, какие координаты есть у каждого сайта.

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Вышеуказанные координаты - это обычные числа, которые не могут дать вам никаких интуитивных сведений о том, где находятся эти стартовые площадки. Если вы хорошо разбираетесь в географии, вы можете интерпретировать эти цифры прямо в уме. Если нет - тоже ничего страшного. Давайте визуализируем эти места, нанеся их на карту.

Сначала нам нужно создать объект фолиума `Map`, начальным центром которого будет Космический центр НАСА имени Джонсона в Хьюстоне, штат Техас.

In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Мы можем использовать `folium.Circle`, чтобы добавить выделенную область круга с текстовой меткой на определенную координату. Например,

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

и вы найдете маленький желтый кружок рядом с городом Хьюстон. Вы можете увеличить масштаб, чтобы увидеть более крупный круг.

Теперь добавим круг для каждого места запуска в кадре данных `launch_sites`.

_TODO:_ Создайте и добавьте `folium.Circle` и `folium.Marker` для каждого места запуска на карте сайта

An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
#site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Создаем карту с фокусом на все места запуска
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Проходим по каждому месту запуска и добавляем круг и маркер
for index, launch_site in launch_sites_df.iterrows():
    # Координаты места запуска
    coordinate = [launch_site['Lat'], launch_site['Long']]

    # Создаем круг вокруг места запуска
    folium.Circle(
        coordinate,
        radius=1000,  # Радиус в метрах
        color='#d35400',  # Цвет круга
        fill=True
    ).add_child(folium.Popup(launch_site['Launch Site'])).add_to(site_map)

    # Создаем маркер для места запуска
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site['Launch Site'],
        )
    ).add_to(site_map)

# Отображаем карту
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Теперь вы можете изучить карту, увеличивая/уменьшая отмеченные области
и попытаться ответить на следующие вопросы:
- Все ли места запуска находятся вблизи линии экватора?
- Все ли места запуска находятся в непосредственной близости от побережья?

Также постарайтесь объяснить свои выводы.

# Задание 2: Отметьте на карте успешные/неудачные запуски для каждого участка

Далее попробуем улучшить карту, добавив результаты запусков для каждого сайта, и посмотрим, какие сайты имеют высокий процент успешных запусков.
Напомним, что в кадре данных spacex_df содержатся подробные записи о запусках, а столбец `class` указывает, был ли этот запуск успешным или нет

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Далее создадим маркеры для всех записей о запуске.
Если запуск был успешным `(class=1)`, то мы используем зеленый маркер, а если запуск был неудачным, то мы используем красный маркер `(class=0)`.

Обратите внимание, что запуск происходит только в одном из четырех мест запуска, а значит, многие записи о запуске будут иметь одинаковые координаты. Кластеры маркеров могут быть хорошим способом упростить карту, содержащую множество маркеров с одинаковыми координатами.

Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


In [11]:
# Вывод всех имен столбцов в DataFrame
print(launch_sites_df.columns)


Index(['Launch Site', 'Lat', 'Long'], dtype='object')


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Создаем новый столбец 'marker_color' в DataFrame launch_sites_df
# Используем значения из столбца 'class' для назначения цвета маркера
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Просмотрим результаты
spacex_df[['Launch Site', 'Lat', 'Long', 'class', 'marker_color']].tail(10)



,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [13]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ Для каждого результата запуска в кадре данных `spacex_df` добавьте `folium.Marker` в `marker_cluster`.

In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
# Create a Marker object with its coordinate
# And customize the Marker's icon property to indicate if this launch was successful or failed
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=record['Launch Site']
    ).add_to(marker_cluster)

# Display map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


По цветовым меткам в скоплениях маркеров вы сможете легко определить, какие места запуска имеют относительно высокий процент успеха.

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Теперь увеличьте масштаб стартовой площадки и изучите ее близость, чтобы увидеть, легко ли найти железную дорогу, шоссе, береговую линию и т. д. Подведите мышь к этим точкам и отметьте их координаты (показаны слева вверху), чтобы определить расстояние до места запуска.

Вы можете рассчитать расстояние между двумя точками на карте на основе их значений `Lat` и `Long`, используя следующий метод:

In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Отметьте точку на ближайшем побережье с помощью MousePosition и рассчитайте расстояние между ней и местом запуска.

In [17]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

После получения координат создайте `folium.Marker`, чтобы показать расстояние.

In [18]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Определите координаты ближайшего побережья (замените на ваши реальные координаты)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Расчет расстояния между стартовой площадкой и побережьем
launch_site_lat = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].values[0]
launch_site_lon = spacex_df.loc[spacex_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].values[0]
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Создание маркера на побережье, который отображает расстояние
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
        )
    )

# Добавляем маркер на карту
site_map.add_child(distance_marker)
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Координаты стартовой площадки и побережья
launch_site_coordinates = [launch_site_lat, launch_site_lon]
coastline_coordinates = [coastline_lat, coastline_lon]

# Создаем PolyLine между стартовой площадкой и побережьем
lines = folium.PolyLine(locations=[launch_site_coordinates, coastline_coordinates], weight=1)

# Добавляем линию на карту
site_map.add_child(lines)
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


Нанеся линии расстояний на близлежащие объекты, вы сможете легко ответить на следующие вопросы:
- Находятся ли стартовые площадки в непосредственной близости от железных дорог?
- Находятся ли стартовые площадки в непосредственной близости от автомобильных дорог?
- Находятся ли места запуска в непосредственной близости от береговой линии?
- Находятся ли космодромы на определенном расстоянии от городов?

Также постарайтесь объяснить свои выводы.

# Следующие шаги:

Теперь вы обнаружили множество интересных фактов, связанных с расположением стартовых площадок, используя фолиум, причем в очень интерактивной форме. Далее вам нужно будет построить приборную панель с помощью Ploty Dash на основе подробных записей о запусках.

## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
